# Minimum Coin Combination
You are given an array of coin values and a target amount of money. Return the minimum number of coins needed to total the target amount. If this isn't possible, return ‐1. You may assume there's an unlimited supply of each coin.

**Example 1:**
```python
Input: coins = [1, 2, 3], target = 5
Output: 2
```
Explanation: Use one 2-dollar coin and one 3-dollar coin to make 5 dollars.

**Example 2:**
```python
Input: coins = [2, 4], target = 5
Output: -1
```

## Intuition - Top-Down Approach

In this problem, there's no restriction on the number of coins we can use, which makes a brute-force approach that tries every possible coin combination infeasible due to the infinite number of possible combinations. This indicates the need for a more efficient method.

Consider the coin set `[1, 2, 3]` and a target of `5`.  
If we use a `3`-dollar coin, then we only need `2` more dollars to reach `5`. This gives us a new target: finding the fewest number of coins needed to make `2` dollars. This shows that we've identified subproblems within the main problem, where each subproblem requires finding the minimum number of coins needed for a smaller target. Each coin we use creates a new subproblem.  

A path that results in a target of `0` means the coins used in that path add up to `5`. If the target becomes negative, the path is invalid, so we should stop extending it.

We've observed how new subproblems are created but haven't yet addressed how to solve them efficiently. Each subproblem needs to return the minimum number of coins required to reach its target.

Consider the main problem with a target of `5`. To solve this, we first need to determine the minimum number of coins required for each of its three subproblems (targets `4`, `3`, and `2`). The solution to the main problem is the smallest result among these subproblems, plus `1`, to account for the coin used to create the subproblem. This highlights an **optimal substructure**, allowing us to define the following recurrence relation:

$$
\text{min\_coin\_combinations}(\text{target}) = 1 + \min(\text{min\_coin\_combinations}(\text{target} - \text{coin}_i)) \quad \text{for each } \text{coin}_i \in \text{coins}
$$

---

### Base Case

Naturally, we need a base case for this formula. The base case occurs when the target equals `0`, which is the simplest version of the problem—no coins are needed to meet the target. In this case, we return `0`.

---

### Memoization

Since we might end up solving the same subproblem multiple times, we can use **memoization** to significantly reduce the size of the recursion tree and improve efficiency.


In [1]:
from typing import List, Dict

def min_coin_combination(coins: List[int], target: int) -> int:
    res = top_down_dp(coins, target, {})
    return -1 if res == float('inf') else res

def top_down_dp(coins: List[int], target: int, memo: Dict[int, int]) -> int:
    if target == 0:
        return 0

    if target in memo:
        return memo[target]
    
    min_coins = float('inf')
    for coin in coins:
        if coin <= target:
            min_coins = min(min_coins, 1 + top_down_dp(coins, target - coin, memo))
    
    memo[target] = min_coins
    return memo[target]

### Complexity Analysis

#### Time Complexity

- Without memoization, the time complexity is **O(n<sup>target/m</sup>)**, where `n` denotes the number of coins and `m` denotes the smallest coin value. The recursion tree has a branch factor of `n` because we make a recursive call for up to `n` coins. The depth of the tree is `target/m` because, in the worst case, we continually reduce the target value by the smallest coin.

- With **memoization**, we ensure each subproblem is solved only once. Since there are at most `target` subproblems, and we iterate through all `n` coins for each subproblem, the time complexity is **O(target × n)**.

---

#### Space Complexity

The space complexity is **O(target)** because, while the maximum depth of the recursive call stack is only `target/m`, the memoization array stores up to `target` key-value pairs.


## Intuition - Bottom-Up Approach

We can convert our top-down solution to a bottom-up one by translating the memoization array into a DP (dynamic programming) array.

First, let's examine the values stored in our memoization array, as shown in the following code snippet from the top-down implementation:

```python
for coin in coins:
    if coin <= target:
        min_coins = min(min_coins, 1 + top_down_dp(coins, target - coin, memo))

memo[target] = min_coins
```

Translating this logic into a DP array gives us the following code:
```python
for coin in coins:
    if coin <= target:
        dp[target] = min(dp[target], 1 + dp[target - coin])
```

This snippet only includes the calculation for a single target value. In our top-down solution, this calculation is repeated for every target value, starting from the initial target and moving down to the base case (`target == 0`).

In the bottom-up solution, we reverse this order by starting with the base case and working our way up to the target value (hence the name "bottom-up"). This is necessary because our DP array calculation depends on the values of smaller targets. Therefore, we must compute the solutions for smaller targets first.

This can be achieved using a `for` loop that iterates from `1` to `target` (starting at `1`, since the base case of `0` is already initialized):

```python
for t in range(1, target + 1):
    for coin in coins:
        if coin <= t:
            dp[t] = min(dp[t], 1 + dp[t - coin])
```

Once this loop completes, the solution to the problem will be stored in `dp[target]`.

In [2]:
from typing import List, Dict

def min_coin_combination(coins: List[int], target: int) -> int:
    dp=[float('inf')] * (target + 1)
    dp[0] = 0

    for t in range(1, target + 1):
        for coin in coins:
            if coin <= t:
                dp[t] = min(dp[t], 1 + dp[t - coin])
    
    return dp[target] if dp[target] != float('inf') else -1

### Complexity Analysis

#### Time Complexity

The time complexity is O(target * n) because we loop through all n coins for each value between 1 and target.
 
---

#### Space Complexity

The space complexity is **O(target)** due to the space occupied by the DP array, which is of size target + 1.